# Crypto Arbitrage
An app to show the difference in price for Bitcoin (BTC) on two exchanges: Bitstamp and Coinbase

In [1]:
import pandas as pd
from pathlib import Path

from datetime import timedelta

import ipywidgets as widgets
from IPython.display import display

import collection
import vizualization as vz


In [2]:
output = widgets.Output()
display(output)

Output()

## Data Collection
Historic trade data imported from CSV file in the Resources folder.
*Timestamp, Open, High, Low, Close, BTC Volume, USD Volume, Weighted Price* is collected from two exchanges: **Bitstamp** and **Coinbase**.

In [3]:
tab_list = ['Bitstamp', 'Coinbase']
bitstamp_summary_output = widgets.Output(layout={'border': '1px solid black'})
coinbase_summary_output = widgets.Output(layout={'border': '1px solid black'})
tab = widgets.Tab()
tab.children = [bitstamp_summary_output, coinbase_summary_output]
for i, t in enumerate(tab_list):
    tab.set_title(i, t)
tab

In [4]:
# Instantiate Data_Container class for Bitstamp
bitstamp = collection.Data_Container()
bitstamp.df = bitstamp.read_csv(Path("Resources/bitstamp.csv"))
bitstamp.clean_data()

# Slice the DataFrames for the timestamp and 'Close' price 
bitstamp.slice_data()

# Generate the summary statistics for the DataFrame
bitstamp_summary = bitstamp.df.describe(include='all')

with bitstamp_summary_output:
    display(bitstamp_summary)

In [5]:
# Instantiate Data_Container class for Coinbase
coinbase = collection.Data_Container()
coinbase.df =  coinbase.read_csv(Path("Resources/coinbase.csv"))
coinbase.clean_data()

# Slice the DataFrames for the timestamp and 'Close' price 
coinbase.slice_data()

# Generate the summary statistics for the DataFrame
coinbase_summary = coinbase.df.describe(include='all')

with coinbase_summary_output:
    display(coinbase_summary)


In [6]:
main_df = pd.DataFrame(data=[bitstamp.sliced, coinbase.sliced], index=['Bitstamp', 'Coinbase']).T
# display(main_df)

## Analysis
Calculate the difference in price *(spread)* between the two exchanges and the *percent return* for each **close price**.

In [7]:
# Calculate the spread between the two exchanges and add column to the DataFrame
main_df['Spread'] = (main_df['Bitstamp'] - main_df['Coinbase']).abs()

In [8]:
# Function to calculate the percentage of the arbitrage spread
def calc_arbitrage_percent(df):
    if df['Bitstamp'] < df['Coinbase']:
        return 100 * df['Spread'] / df['Bitstamp']
    elif df['Bitstamp'] > df['Coinbase']:
        return 100 * df['Spread'] / df['Coinbase']
    else:
        return 0.00

In [9]:
# Apply the function to calculate arbitrage spread and add the column to the DataFrame
main_df['Return %'] = main_df.apply(calc_arbitrage_percent, axis=1)

In [23]:
# Review the spread return DataFrame
display(main_df)

,Bitstamp,Coinbase,Spread,Return %
Timestamp,,,,
2018-01-01 00:00:00,13646.48,13608.49,37.99,0.279164
2018-01-01 00:01:00,13658.75,13601.66,57.09,0.419728
2018-01-01 00:02:00,13610.22,13580.00,30.22,0.222533
2018-01-01 00:03:00,13639.09,13550.34,88.75,0.654965
2018-01-01 00:04:00,13620.00,13583.44,36.56,0.269151
...,...,...,...,...
2018-03-31 23:55:00,6922.56,6930.00,7.44,0.107475
2018-03-31 23:56:00,6920.32,6930.01,9.69,0.140022
2018-03-31 23:57:00,6934.72,6933.91,0.81,0.011682


In [ ]:
## Graph Functions and Widgets

In [11]:
# Function to slice DataFrame to 1 Day centered on the value passed to the function
def gen_sliced_df(date_picked):
    slice_start_date = bitstamp.sliced.index[date_picked] - timedelta(days=.5)
    slice_end_date = slice_start_date + timedelta(days=1)

    sliced_df = main_df.loc[slice_start_date:slice_end_date]

    return sliced_df

In [12]:
# Slider to adjust the plot's vertical scale
scale = widgets.FloatRangeSlider(
    value = [.75*min(main_df['Bitstamp']), 1.25*max(main_df['Bitstamp'])],
    min=.75*min(main_df['Bitstamp']), 
    max=1.25*max(main_df['Bitstamp']),
    description='Scale',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=False,
    readout_format='d',
    # layout=widgets.Layout(height='350px'),
    layout=widgets.Layout(height='100%'),
)

In [13]:
# Checkboxes to show/hide the data for each exchange
bitstamp_checkbox = widgets.Checkbox(
    value=True,
    description='Bitstamp',
    disabled=False
)

coinbase_checkbox = widgets.Checkbox(
    value=True,
    description='Coinbase',
    disabled=False
)


In [14]:
# Slider to adjust the plot's horizontal scale
date_slicer = widgets.IntRangeSlider(
    value=[0, len(main_df.index)-1],
    min=0,
    max=len(main_df.index)-1,
    step=1,
    description='Date Range',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


In [15]:
# Sliders to pick center date for each analysis window tab
date_picker_1 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    # layout=widgets.Layout(width=f'900px')
    layout=widgets.Layout(width=f'100%')
    )

date_picker_2 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    # layout=widgets.Layout(width=f'900px')
    layout=widgets.Layout(width=f'100%')
    )

date_picker_3 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    # layout=widgets.Layout(width=f'900px')
    layout=widgets.Layout(width=f'100%')
    )


In [16]:
# Text box to display the date picked for each tab
date_picked_1 = widgets.Text(
    f"{main_df.index[date_picker_1.value]}",
    )

date_picked_2 = widgets.Text(
    f"{main_df.index[date_picker_2.value]}",
    )

date_picked_3 = widgets.Text(
    f"{main_df.index[date_picker_3.value]}",
    )


In [ ]:
## Plots

In [17]:
# Main plot
def main_plotter(
    range_slider, 
    date_slicer, 
    bitstamp_checkbox, 
    coinbase_checkbox, 
    ):
    """
    A helper function to make a graph.
    """
    vz.main_plotter(
        main_df, 
        range_slider, 
        date_slicer, 
        bitstamp_checkbox, 
        coinbase_checkbox, 
        )

main_plot = widgets.interactive_output(
    main_plotter, 
    {
        'range_slider': scale,
        'date_slicer': date_slicer,
        'bitstamp_checkbox': bitstamp_checkbox,
        'coinbase_checkbox': coinbase_checkbox,
        }
    )


In [18]:
# Single day plot for each analysis tab
def day_plotter(date_picker):
    vz.day_plotter(main_df, date_picker)
    
day_out_1 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_1,
    }
)

day_out_2 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_2,
    }
)

day_out_3 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_3,
    }
)


In [19]:
def arbitrage_summary(target_date_index):
    sliced_df = gen_sliced_df(target_date_index)
    vz.arbitrage_summary(sliced_df['Spread'])

arbitrage_out_1 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_1
    }
)

arbitrage_out_2 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_2
    }
)

arbitrage_out_3 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_3
    }
)


In [20]:
tab_1_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_1, 
                date_picked_1
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_1, 
                day_out_1
            ]
        ), 
    ]
) 

tab_2_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_2, 
                date_picked_2
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_2, 
                day_out_2
            ]
        ), 
    ]
) 

tab_3_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_3, 
                date_picked_3
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_3, 
                day_out_3
            ]
        ), 
    ]
) 

tab_titles = ['First Tab', 'Second Tab', 'Third Tab']

children = [tab_1_container, tab_2_container, tab_3_container]
tab = widgets.Tab()
for i, t in enumerate(tab_titles):
    tab.set_title(i, t)
tab.children = children


In [ ]:
# Graph

In [21]:
vbox = widgets.VBox(
    [
        widgets.HBox([date_slicer, widgets.VBox([bitstamp_checkbox, coinbase_checkbox])]), 
        widgets.HBox([scale, main_plot]), 
        tab,
        ],
    )

with output:
    display(vbox)